# Getting started with flowline models: idealized experiments

In this notebook we are going to explore the basic functionalities of OGGM flowline model(s). For this purpose we are going to use simple and "idealized" glaciers models and compute them with simple linear mass balance profiles. 

In [ ]:
# Plotting libraries and plot style
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('notebook')
sns.set_style('ticks')

# Scientific packages
import numpy as np

# Constants
from oggm import cfg
cfg.initialize_minimal()

# Mass-balance model
from oggm.core.massbalance import LinearMassBalance
# There are several numerical implementations in OGGM core. We use the "FluxBasedModel"
from oggm.core.flowline import FluxBasedModel as FlowlineModel
# Glacier shape
from oggm.core.flowline import RectangularBedFlowline

# OGGM Edu helper functions
import oggm_edu as edu

## Basics

First we set-up a simple run with a constant linear bedrock:

### Glacier bed

In [ ]:
# define horizontal resolution of the model:
# nx: number of grid points
# map_dx: grid point spacing in meters
nx = 200
map_dx = 100

In [ ]:
# define glacier top and bottom altitudes in meters
top = 3400
bottom = 1400

In [ ]:
# create a linear bedrock profile from top to bottom
bed_h, surface_h = edu.define_linear_bed(top, bottom, nx)

In [ ]:
# calculate the distance from the top to the bottom of the glacier in km
distance_along_glacier = edu.distance_along_glacier(nx, map_dx)

In [ ]:
# plot the glacier bedrock profile and the initial glacier surface
plt.plot(distance_along_glacier, surface_h, label='Initial glacier')
edu.plot_xz_bed(x=distance_along_glacier, bed=bed_h);

Now we have to decide how wide our glacier is, and what the *shape* of its bed is. For a start, we will use a rectangular "u-shaped" bed (see the [documentation](http://docs.oggm.org/en/stable/ice-dynamics.html#rectangular)), with a constant width of 300m:

In [ ]:
initial_width = 300  # width in meters
# Now describe the widths in "grid points" for the model, based on grid point spacing map_dx
widths = np.zeros(nx) + initial_width/map_dx
# Define our bed
init_flowline = RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h, widths=widths, map_dx=map_dx)

The init_flowline variable now contains all geometrical information needed by the model. It can give access to some attributes, which are quite useless for a non-existing glacier: 

In [ ]:
print('Glacier length:', init_flowline.length_m)
print('Glacier area:', init_flowline.area_km2)
print('Glacier volume:', init_flowline.volume_km3)

### Mass balance

Then we will need a mass balance model. In our case this will be a simple linear mass balance, defined by the equilibrium line altitude (ELA) and an altitude gradient (in [mm yr$^{-1}$ m$^{-1}$]):

In [ ]:
# ELA at 3000m a.s.l., gradient 4 mm yr -1 m-1
ELA = 3000  # equilibrium line altitude in meters above sea level
altgrad = 4  # altitude gradient in mm/m
mb_model = LinearMassBalance(ELA, grad=altgrad)

The OGGM mass balance model now computes the mass balance for any given altitude (in units meters of ice per time [m s$^{-1}$], which is simpler for the ice dynamics model). Let us compute the *annual* mass balance along the glacier profile:

In [ ]:
annual_mb = mb_model.get_annual_mb(surface_h) * cfg.SEC_IN_YEAR

In [ ]:
# Plot it
plt.plot(annual_mb, bed_h, color='C2', label='Mass balance')
plt.xlabel('Annual mass balance (m yr-1)')
plt.ylabel('Altitude (m)')
plt.legend(loc='best')
# Display equilibrium line altitude, where annual mass balance = 0
plt.axvline(x=0, color='k', linestyle='--', linewidth=0.8)
plt.axhline(y=mb_model.ela_h, color='k', linestyle='--', linewidth=0.8);

### Model run

Now that we have all the ingredients to run the model, we just have to initialize it:

In [ ]:
# The model requires the initial glacier bed, a mass balance model, and an initial time (the year y0)
model = FlowlineModel(init_flowline, mb_model=mb_model, y0=0.)

Let's first run the model for one year:

In [ ]:
runtime = 1
model.run_until(runtime)
edu.glacier_plot(x=distance_along_glacier, bed=bed_h, model=model, mb_model=mb_model, init_flowline=init_flowline)

In [ ]:
print('Year:', model.yr)
print('Glacier length (m):', model.length_m)
print('Glacier area (km2):', model.area_km2)
print('Glacier volume (km3):', model.volume_km3)

The modeled 'glacier' fills already the whole bed and its length goes up to the ELA (dashed line), but it is extremely thin. 

We can now run the model for 150 years and see how the output looks like:

In [ ]:
runtime = 150
model.run_until(runtime)
edu.glacier_plot(x=distance_along_glacier, bed=bed_h, model=model, mb_model=mb_model, init_flowline=init_flowline)

Let's print out a few infos about our glacier:

In [ ]:
print('Year:', model.yr)
print('Glacier length (m):', model.length_m)
print('Glacier area (km2):', model.area_km2)
print('Glacier volume (km3):', model.volume_km3)

Note that the model time is now 150 years. Running the model with the same input again, calls the already calculated results but does not execute the method `model.run_until` another time, which safes computational time. 



In [ ]:
model.run_until(150)
print('Year:', model.yr)
print('Glacier length (m):', model.length_m)

If we want to compute longer, we have to set the desired date. Hereby, the model computes only the additional missing years.

In [ ]:
runtime = 500
model.run_until(runtime)
edu.glacier_plot(x=distance_along_glacier, bed=bed_h, model=model, mb_model=mb_model, init_flowline=init_flowline)

In [ ]:
print('Year:', model.yr)
print('Glacier length (m):', model.length_m)
print('Glacier area (km2):', model.area_km2)
print('Glacier volume (km3):', model.volume_km3)

However, it is important to note, that the model will not calculate back in time.
Once calculated for 500 years, the model will not run again for 450 years and remains at 500 years.  Try running the cell below.  Does the output match what you expected?

In [ ]:
model.run_until(450)
print('Year:', model.yr)
print('Glacier length (m):', model.length_m)

It might be useful to store some intermediate steps of the evolution of the glacier.  We make a loop so that the model reports to us several times.

In [ ]:
# Reinitialize the model
model = FlowlineModel(init_flowline, mb_model=mb_model, y0=0.)

# Year 0 to 600 in 5 years step
yrs = np.arange(0, 601, 5) 

# Array to fill with data
nsteps = len(yrs)
length = np.zeros(nsteps)
vol = np.zeros(nsteps)

# Loop over the years
for i, yr in enumerate(yrs):
    model.run_until(yr)
    length[i] = model.length_m
    vol[i] = model.volume_km3

# Store the final results for later use
simple_glacier_h = model.fls[-1].surface_h

We can now plot the evolution of the glacier length and volume with time:

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.plot(yrs, length);
ax1.set_xlabel('Years'); ax1.set_ylabel('Length (m)');
ax2.plot(yrs, vol);
ax2.set_xlabel('Years'); ax2.set_ylabel('Volume (km3)');

The glacier length is a step function in the first year of simulation because above the equilibrium line altitude (ELA), only accumulation takes places and OGGM currently does not differentiate between ice and snow.
After that, the glacier's length remains first constant. This behavior can be explained by the fact that above the ELA, the mass balance is positive, unimportant how large the glacier is. As long as the glacier is not thick enough, no ice remains below the ELA, because the ablation below the ELA is higher than the amount of ice forming the glacier flow. 

In [ ]:
'Glacier length from the top to the equilibrium line altitude ('+str(mb_model.ela_h)+' m) is: {} m'.format(length[1])

After several centuries, the glacier gets in balance with its climate. Its length and volume won't change anymore, if all physical parameters and the climate stay constant.

## A first experiment 

Ok, now we have seen the basics. We will now define a simple experiment, in which we will make the glacier wider at the top (in the accumulation area). This is a common situation for valley glaciers.

In [ ]:
# We copy the widths we defined before
wider_widths = np.copy(widths)
# But we now make our glacier 600 m wide at the top:
new_width = 600 #meters
# Convert the width in meters to width in "grid units", like before, and rewrite the first few points
wider_widths[0:15] = new_width/map_dx
# Define our new bed
wider_flowline = RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h, widths=wider_widths, map_dx=map_dx)

We will now run our model with the new inital conditions (again for 600 years), and store the output in a new variable for comparison:

In [ ]:
# Reinitialize the model with the new input
model = FlowlineModel(wider_flowline, mb_model=mb_model, y0=0.)

# Array to fill with data
nsteps = len(yrs)
length_w = np.zeros(nsteps)
vol_w = np.zeros(nsteps)

# Loop over the years
for i, yr in enumerate(yrs):
    model.run_until(yr)
    length_w[i] = model.length_m
    vol_w[i] = model.volume_km3

# Store the final results for later use
wider_glacier_h = model.fls[-1].surface_h

Compare the results:

In [ ]:
# Plot the final result:
plt.plot(distance_along_glacier, simple_glacier_h, label='Simple glacier')
plt.plot(distance_along_glacier, wider_glacier_h, label='Wider glacier')
# Add the bedrock:
edu.plot_xz_bed(x=distance_along_glacier, bed=bed_h);

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.plot(yrs, length, label='Simple glacier');
ax1.plot(yrs, length_w, label='Wider glacier');
ax1.legend(loc='best')
ax1.set_xlabel('Years')
ax1.set_ylabel('Length (m)');
ax2.plot(yrs, vol, label='Simple glacier');
ax2.plot(yrs, vol_w, label='Wider glacier');
ax2.legend(loc='best')
ax2.set_xlabel('Years')
ax2.set_ylabel('Volume (km3)');

## What's next?

[Back to the table of contents](../welcome.ipynb)